In [2]:
# Example weather station data
#
# 1419408015	0R1,Dn=059D,Dm=066D,Dx=080D,Sn=8.5M,Sm=9.5M,Sx=10.3M
# 1419408016	0R1,Dn=059D,Dm=065D,Dx=078D,Sn=8.5M,Sm=9.5M,Sx=10.3M
# 1419408016	0R2,Ta=13.9C,Ua=28.5P,Pa=889.9H
# 1419408017	0R1,Dn=059D,Dm=064D,Dx=075D,Sn=8.7M,Sm=9.6M,Sx=10.3M
# 1419408018	0R1,Dn=059D,Dm=064D,Dx=075D,Sn=8.9M,Sm=9.6M,Sx=10.3M
# 1419408019	0R1,Dn=059D,Dm=065D,Dx=075D,Sn=8.8M,Sm=9.5M,Sx=10.3M

In [3]:
# Key for measurements:
#
# Sn      Wind speed minimum m/s, km/h, mph, knots #,M, K, S, N
# Sm      Wind speed average m/s, km/h, mph, knots #,M, K, S, N
# Sx      Wind speed maximum m/s, km/h, mph, knots #,M, K, S, N
# Dn      Wind direction minimum deg #, D
# Dm      Wind direction average deg #, D
# Dx      Wind direction maximum deg #, D
# Pa      Air pressure hPa, Pa, bar, mmHg, inHg #, H, P, B, M, I
# Ta      Air temperature °C, °F #, C, F
# Tp      Internal temperature °C, °F #, C, F
# Ua      Relative humidity %RH #, P
# Rc      Rain accumulation mm, in #, M, I
# Rd      Rain duration s #, S
# Ri      Rain intensity mm/h, in/h #, M, I
# Rp      Rain peak intensity mm/h, in/h #, M, I
# Hc      Hail accumulation hits/cm2, hits/in2, hits #, M, I, H
# Hd      Hail duration s #, S
# Hi      Hail intensity hits/cm2h, hits/in2h, hits/ h #, M, I, H
# Hp      Hail peak intensity hits/cm2h, hits/in2h, hits/ h #, M, I, H
# Th      Heating temperature °C, °F #, C, F
# Vh      Heating voltage V #, N, V, W, F2
# Vs      Supply voltage V V
# Vr      3.5 V ref. voltage V V

In [1]:
# Parse a line of weather station data, returning the average wind direction measurement 
#
import re
def parse(line):
    match = re.search("Dm=(\d+)", line)
    if match:
        val = match.group(1)
        return [int(val)]
    return []

In [2]:
from pyspark.streaming import StreamingContext

Spark Context already provided

In [3]:
sc

In [4]:
batch_duration = 1 # DRDDs of 1 sec
ssc = StreamingContext(sc, batch_duration)

In [5]:
lines = ssc.socketTextStream("rtd.hpwren.ucsd.edu", 12028)

In [6]:
vals = lines.flatMap(parse)
win_secs, slide_secs = 10, 5
window = vals.window(win_secs,slide_secs)

In [7]:
def stats(rdd):
    print(rdd.collect())
    if rdd.count() > 0:
        print("max = {} min = {}".format(rdd.max(), rdd.min()))

In [8]:
window.foreachRDD(lambda rdd: stats(rdd))

In [9]:
# will print every 5 seconds (slide_secs)
ssc.start()

[]
[0, 1, 3, 5]
max = 5 min = 0
[0, 1, 3, 5, 4, 350]
max = 350 min = 0
[4, 350, 7, 8, 8, 10, 26, 41, 39, 46]
max = 350 min = 4
[7, 8, 8, 10, 26, 41, 39, 46, 52, 51, 61, 64, 72, 83]
max = 83 min = 7
[52, 51, 61, 64, 72, 83, 70]
max = 83 min = 51
[70, 70, 74, 85, 87, 82, 74]
max = 87 min = 70
[70, 74, 85, 87, 82, 74, 63, 58, 46]
max = 87 min = 46


In [10]:
ssc.stop()